In [151]:
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torch import tensor, nn

In [9]:
data = pd.read_hdf('lstm.h5', 'data_2011')
data

,QUOTE_UNIXTIME,QUOTE_READTIME,QUOTE_DATE,QUOTE_TIME_HOURS,UNDERLYING_LAST,EXPIRE_DATE,EXPIRE_UNIX,DTE,C_DELTA,C_GAMMA,...,P_THETA,P_RHO,P_IV,P_VOLUME,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT,TTM,Moneyness,TIME_IDX,RISK_FREE_RATE
233248,1294088400,2011-01-03 16:00:00,2011-01-03,16.0,1271.36,2012-06-14,1339704000,527.96,1.00000,0.00000,...,-0.06619,-1.44521,0.34339,NaN,546.4,0.430,1.446461,0.570255,0,0.383771
233249,1294088400,2011-01-03 16:00:00,2011-01-03,16.0,1271.36,2012-06-14,1339704000,527.96,1.00000,0.00000,...,-0.06203,-1.28876,0.35148,NaN,571.4,0.449,1.446461,0.550591,0,0.383771
233250,1294088400,2011-01-03 16:00:00,2011-01-03,16.0,1271.36,2012-06-14,1339704000,527.96,1.00000,0.00000,...,-0.08167,-2.03089,0.32251,0.0,471.4,0.371,1.446461,0.629247,0,0.383771
233251,1294088400,2011-01-03 16:00:00,2011-01-03,16.0,1271.36,2012-06-14,1339704000,527.96,1.00000,0.00000,...,-0.08720,-2.26867,0.31659,0.0,446.4,0.351,1.446461,0.648911,0,0.383771
233252,1294088400,2011-01-03 16:00:00,2011-01-03,16.0,1271.36,2012-06-14,1339704000,527.96,0.94555,0.00045,...,-0.09182,-2.51553,0.30917,0.0,421.4,0.331,1.446461,0.668575,0,0.383771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481625,1325278800,2011-12-30 16:00:00,2011-12-30,16.0,1257.91,2012-02-16,1329426000,48.00,1.00000,0.00000,...,-0.06954,-0.02534,0.42059,0.0,352.9,0.281,0.131507,0.719447,251,0.043920
481626,1325278800,2011-12-30 16:00:00,2011-12-30,16.0,1257.91,2012-02-16,1329426000,48.00,1.00000,0.00000,...,-0.05944,-0.01987,0.43147,10.0,372.9,0.296,0.131507,0.703548,251,0.043920
481627,1325278800,2011-12-30 16:00:00,2011-12-30,16.0,1257.91,2012-02-16,1329426000,48.00,1.00000,0.00000,...,-0.05497,-0.01766,0.43823,1.0,382.9,0.304,0.131507,0.695598,251,0.043920
481628,1325278800,2011-12-30 16:00:00,2011-12-30,16.0,1257.91,2012-02-16,1329426000,48.00,1.00000,0.00000,...,-0.07038,-0.02431,0.43345,10.0,362.9,0.288,0.131507,0.711498,251,0.043920


In [10]:
underlying = pd.read_hdf('lstm.h5', 'underlying_2011')
underlying

,QUOTE_DATE,UNDERLYING_LAST,RETURNS
TIME_IDX,,,
0,2011-01-03,1271.36,NaN
1,2011-01-04,1270.37,-0.99
2,2011-01-05,1276.47,6.10
3,2011-01-06,1273.88,-2.59
4,2011-01-07,1271.60,-2.28
...,...,...,...
247,2011-12-23,1265.21,11.20
248,2011-12-27,1265.60,0.39
249,2011-12-28,1249.37,-16.23


In [54]:
underlying.fillna(0, inplace=True)
underlying

,QUOTE_DATE,UNDERLYING_LAST,RETURNS
TIME_IDX,,,
0,2011-01-03,1271.36,0.00
1,2011-01-04,1270.37,-0.99
2,2011-01-05,1276.47,6.10
3,2011-01-06,1273.88,-2.59
4,2011-01-07,1271.60,-2.28
...,...,...,...
247,2011-12-23,1265.21,11.20
248,2011-12-27,1265.60,0.39
249,2011-12-28,1249.37,-16.23


In [18]:
date = underlying.iloc[99]['QUOTE_DATE']
data[data['QUOTE_DATE'] >= date]

,QUOTE_UNIXTIME,QUOTE_READTIME,QUOTE_DATE,QUOTE_TIME_HOURS,UNDERLYING_LAST,EXPIRE_DATE,EXPIRE_UNIX,DTE,C_DELTA,C_GAMMA,...,P_THETA,P_RHO,P_IV,P_VOLUME,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT,TTM,Moneyness,TIME_IDX,RISK_FREE_RATE
326961,1306353600,2011-05-25 16:00:00,2011-05-25,16.0,1320.44,2011-07-14,1310673600,50.0,0.63492,0.00462,...,-0.33227,-0.69767,0.15968,115.0,30.4,0.023,0.136986,0.976947,99,0.064401
326962,1306353600,2011-05-25 16:00:00,2011-05-25,16.0,1320.44,2011-07-14,1310673600,50.0,0.61279,0.00480,...,-0.33594,-0.74222,0.15686,1.0,25.4,0.019,0.136986,0.980734,99,0.064401
326963,1306353600,2011-05-25 16:00:00,2011-05-25,16.0,1320.44,2011-07-14,1310673600,50.0,0.59000,0.00492,...,-0.33933,-0.78972,0.15517,NaN,20.4,0.015,0.136986,0.984520,99,0.064401
326964,1306353600,2011-05-25 16:00:00,2011-05-25,16.0,1320.44,2011-07-14,1310673600,50.0,0.56581,0.00499,...,-0.33940,-0.83780,0.15179,3.0,15.4,0.012,0.136986,0.988307,99,0.064401
326965,1306353600,2011-05-25 16:00:00,2011-05-25,16.0,1320.44,2011-07-14,1310673600,50.0,0.43332,0.00530,...,-0.32655,-1.11466,0.13713,1.0,9.6,0.007,0.136986,1.007240,99,0.064401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481625,1325278800,2011-12-30 16:00:00,2011-12-30,16.0,1257.91,2012-02-16,1329426000,48.0,1.00000,0.00000,...,-0.06954,-0.02534,0.42059,0.0,352.9,0.281,0.131507,0.719447,251,0.043920
481626,1325278800,2011-12-30 16:00:00,2011-12-30,16.0,1257.91,2012-02-16,1329426000,48.0,1.00000,0.00000,...,-0.05944,-0.01987,0.43147,10.0,372.9,0.296,0.131507,0.703548,251,0.043920
481627,1325278800,2011-12-30 16:00:00,2011-12-30,16.0,1257.91,2012-02-16,1329426000,48.0,1.00000,0.00000,...,-0.05497,-0.01766,0.43823,1.0,382.9,0.304,0.131507,0.695598,251,0.043920
481628,1325278800,2011-12-30 16:00:00,2011-12-30,16.0,1257.91,2012-02-16,1329426000,48.0,1.00000,0.00000,...,-0.07038,-0.02431,0.43345,10.0,362.9,0.288,0.131507,0.711498,251,0.043920


In [64]:
data[columns].iloc[3][0]

/var/folders/4h/wb36ylt17sj6v882tp7lx6140000gn/T/ipykernel_68142/2296864654.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data[columns].iloc[3][0]


0.0

In [267]:
columns = ['C_LAST', 'UNDERLYING_LAST', 'STRIKE', 'TTM', 'RISK_FREE_RATE', 'TIME_IDX', 'QUOTE_DATE']

class LSTMDataset(Dataset):
    def __init__(self, samples: pd.DataFrame, underlying: pd.DataFrame, timesteps: int) -> None:
        self.samples = samples[samples['QUOTE_DATE'] >= underlying.iloc[timesteps - 1]['QUOTE_DATE']][columns]
        self.underlying = underlying
        self.timesteps = timesteps
        super().__init__()
    
    def __len__(self) -> int:
        return len(self.samples)
    
    def __getitem__(self, index: int) -> tuple[tensor]:
        sample = self.samples.iloc[index]
        time_idx = sample['TIME_IDX'] + 1
        underlying = self.underlying.iloc[time_idx - self.timesteps : time_idx]
        return (tensor(underlying['RETURNS'].to_numpy()), tensor(sample.iloc[1:5]), tensor(sample.iloc[0]))


In [291]:
dataset = LSTMDataset(data, underlying, 5)
len(dataset)

244462

In [292]:
lstm_dataloader = DataLoader(dataset, 1, True)

In [293]:
next(iter(lstm_dataloader))

/var/folders/4h/wb36ylt17sj6v882tp7lx6140000gn/T/ipykernel_68142/3720823502.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return (tensor(underlying['RETURNS'].to_numpy()), tensor(sample.iloc[1:5]), tensor(sample.iloc[0]))


[tensor([[  0.1500,   5.0300, -14.5600,   7.3700,  17.9200]],
        dtype=torch.float64),
 tensor([[1.3304e+03, 1.9000e+03, 1.1534e+00, 2.8585e-01]], dtype=torch.float64),
 tensor([0.4500], dtype=torch.float64)]

In [256]:
class LstmMLP(nn.Module):

    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.lstm_bn = nn.BatchNorm1d(1)
        self.lstm = nn.LSTM(
            input_size=1, 
            hidden_size=8, 
            num_layers=3,
            batch_first=True
        )
        self.mlp_bn = nn.BatchNorm1d(12)
        self.mlp = nn.Sequential(
            nn.Linear(12, 200),
            nn.LeakyReLU(0.05),
            nn.Linear(200, 200),
            nn.LeakyReLU(0.05),
            nn.Linear(200, 200),
            nn.LeakyReLU(0.05),
            nn.Linear(200, 1),
        )

    def forward(self, x):
        r, s = x
        r = r.unsqueeze(dim=2)
        r = self.lstm_bn(r.permute(0, 2, 1)).permute(0, 2, 1)
        lstm_out, _ = self.lstm(r)
        mlp_in = cat([lstm_out[:,-1,:], s], dim=1)
        mlp_in = self.mlp_bn(mlp_in)
        out = self.mlp(mlp_in)
        return out

        

In [255]:
data['C_LAST'].max()

1231.0

In [198]:
r, s, t = next(iter(lstm_dataloader))
x = (r.float(),s.float())

/var/folders/4h/wb36ylt17sj6v882tp7lx6140000gn/T/ipykernel_68142/2612822343.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return (tensor(underlying['RETURNS']), tensor(sample.iloc[1:5]), tensor(sample.iloc[0]))


In [250]:
t

tensor([  0.0000,   0.0000, 247.2000,   0.0000,   0.0000], dtype=torch.float64)

In [257]:
network = LstmMLP()
network

LstmMLP(
  (lstm_bn): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(1, 8, num_layers=3, batch_first=True)
  (mlp_bn): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (mlp): Sequential(
    (0): Linear(in_features=12, out_features=200, bias=True)
    (1): LeakyReLU(negative_slope=0.05)
    (2): Linear(in_features=200, out_features=200, bias=True)
    (3): LeakyReLU(negative_slope=0.05)
    (4): Linear(in_features=200, out_features=200, bias=True)
    (5): LeakyReLU(negative_slope=0.05)
    (6): Linear(in_features=200, out_features=1, bias=True)
  )
)

In [258]:
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR
from torch import sqrt

optimizer = Adam(network.parameters(), lr=0.057)
scheduler = ExponentialLR(optimizer=optimizer, gamma=0.91)
criterion = nn.MSELoss()


In [282]:
epochs = 30
losses = []

for e in range(epochs):
    total_loss = 0

    network.train()
    
    for idx, (r, s, t) in enumerate(lstm_dataloader):
        optimizer.zero_grad()
        x = (r.float(), s.float())
        pred = network(x)
        loss = sqrt(criterion(pred.squeeze(), t.float()))
        
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
    mean_loss = total_loss / idx + 1
    losses.append(mean_loss)
    print(f"Epoch {e} loss: {mean_loss}")
    scheduler.step()

/var/folders/4h/wb36ylt17sj6v882tp7lx6140000gn/T/ipykernel_68142/3720823502.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return (tensor(underlying['RETURNS'].to_numpy()), tensor(sample.iloc[1:5]), tensor(sample.iloc[0]))


Epoch 0 loss: 155.73120525905065
Epoch 1 loss: 153.0899223600115
Epoch 2 loss: 151.84334877559118
Epoch 3 loss: 150.26861667633057
Epoch 4 loss: 148.48238590785436
Epoch 5 loss: 146.84991427830286
Epoch 6 loss: 146.7036008153643
Epoch 7 loss: 145.18958255222864
Epoch 8 loss: 143.6244468007769
Epoch 9 loss: 142.38821159090315
Epoch 10 loss: 141.6579667500087
Epoch 11 loss: 141.05778312683105
Epoch 12 loss: 140.01251561301095
Epoch 13 loss: 138.98520864759172
Epoch 14 loss: 139.18703086035592
Epoch 15 loss: 137.59717995779855
Epoch 16 loss: 137.29756007875716
Epoch 17 loss: 136.68551479067122
Epoch 18 loss: 136.57212162017822
Epoch 19 loss: 136.1809913090297
Epoch 20 loss: 135.3025972502572
Epoch 21 loss: 134.95983382633753
Epoch 22 loss: 135.60753059387207
Epoch 23 loss: 134.3867722920009
Epoch 24 loss: 133.71091522489274
Epoch 25 loss: 133.0701619556972
Epoch 26 loss: 132.19117001124792
Epoch 27 loss: 129.61930056980677
Epoch 28 loss: 129.17967898505074
Epoch 29 loss: 126.7064938545227

In [290]:
r, s, t = next(iter(lstm_dataloader))
x = (r.float(),s.float())
print(network(x))
print(t)

/var/folders/4h/wb36ylt17sj6v882tp7lx6140000gn/T/ipykernel_68142/3720823502.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return (tensor(underlying['RETURNS'].to_numpy()), tensor(sample.iloc[1:5]), tensor(sample.iloc[0]))


tensor([[ 66.9982],
        [ 94.6239],
        [ 12.9070],
        ...,
        [124.5599],
        [ 13.4784],
        [  1.9694]], grad_fn=<AddmmBackward0>)
tensor([  0.0000,   0.0000,   2.0000,  ..., 117.0000,   0.0000,   0.1500],
       dtype=torch.float64)


In [288]:
epochs = 50
losses = []

for e in range(epochs):
    total_loss = 0

    network.train()
    
    for idx, (r, s, t) in enumerate(lstm_dataloader):
        optimizer.zero_grad()
        x = (r.float(), s.float())
        pred = network(x)
        loss = sqrt(criterion(pred.squeeze(), t.float()))
        
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
    mean_loss = total_loss / idx + 1
    losses.append(mean_loss)
    print(f"Epoch {e} loss: {mean_loss}")
    scheduler.step()

/var/folders/4h/wb36ylt17sj6v882tp7lx6140000gn/T/ipykernel_68142/3720823502.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return (tensor(underlying['RETURNS'].to_numpy()), tensor(sample.iloc[1:5]), tensor(sample.iloc[0]))


Epoch 0 loss: 125.82235145568848
Epoch 1 loss: 124.50708879743304
Epoch 2 loss: 124.12376723970685
Epoch 3 loss: 122.9707910673959
Epoch 4 loss: 122.3268528665815
Epoch 5 loss: 122.19411509377616
Epoch 6 loss: 121.72704819270542
Epoch 7 loss: 121.1847769192287
Epoch 8 loss: 120.53908429827008
Epoch 9 loss: 120.70496756689889
Epoch 10 loss: 120.43752023151943
Epoch 11 loss: 120.03800691877093
Epoch 12 loss: 120.14976011003766
Epoch 13 loss: 119.21041440963745
Epoch 14 loss: 119.38032872336251
Epoch 15 loss: 119.21805906295776
Epoch 16 loss: 119.51594155175346
Epoch 17 loss: 118.66089630126953
Epoch 18 loss: 118.76171629769462
Epoch 19 loss: 118.81126948765346
Epoch 20 loss: 118.00251633780343
Epoch 21 loss: 118.64788355146136
Epoch 22 loss: 118.4025434766497
Epoch 23 loss: 118.45843914576939
Epoch 24 loss: 118.13575492586408
Epoch 25 loss: 118.48916530609131
Epoch 26 loss: 117.63946955544608
Epoch 27 loss: 117.81572839191982
Epoch 28 loss: 118.19327769960675
Epoch 29 loss: 117.979277610

KeyboardInterrupt: 